In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm, colors
import glob

import hoomd
import gc
import warnings
import copy
import pathlib

import os
import shutil

import signac

In [3]:
from monk import nb, prep, pair, render, utils, workflow, grid

parent = pathlib.Path(os.getcwd()).parent / "config.yaml"
config = workflow.get_config(parent.as_posix())
parent, config

(PosixPath('/home/ian/Projects/work/monk/workflows/2d-osc-shear/config.yaml'),
 {'root': '/media/ian/Data2/monk/2d-osc-shear',
  'origin': '/media/ian/Data2/monk/2d-esl'})

In [4]:
project: signac.Project = signac.init_project(name="2d-osc-shear", root=config['root'])
project.doc

{'avail_seed': 6, 'dt': 0.005, 'step_unit': 200, 'equil_time': 100, 'min_periods': 20, 'dumps': 40, 'period_times': [30.0, 100.0, 300.0, 1000.0], 'max_shears': [0.01, 0.02, 0.03, 0.05, 0.08, 0.12], '_status': {}}

In [5]:
for job in project:
    print(job)

634e14bc75836df75158fb15f379d8bb
cfa1e4c0007fd65d020f809715a6f0b4
ecce68c50e28a33684826f28780bf6e9
1083a2a09eec2c6cf215ebc7a9706cac
6976a7c10ada4159010403f8027f3ea5
9158e73b2e4c9fd1a565e1e5d1bc3af5


In [6]:
origin: signac.Project = signac.get_project(root=config['origin'])
origin.doc

{'avail_seed': 11, 'dt': 0.005, 'step_unit': 1000, 'temp_steps': 20, 'equil_time': 40, 'run_time': 10, 'max_alpha_time': 1000, 'alpha_iters': 10, '_status': {}}

In [9]:
tgs = {}
for job in origin:
    try:
        tg = job.data["results/tg"]
        if job.sp["replica"] == 0:
            tgs[job.sp["pot"]] = tg
            print(tg)
    except KeyError:
        pass
tgs

0.19836457263395846
0.12345081002164043


{'KA_LJ': 0.19836457263395846, 'KA_WCA': 0.12345081002164043}

In [10]:
# only set once
if "avail_seed" not in project.doc:
    project.doc["avail_seed"] = 0

In [11]:
project.doc["dt"] = 0.005
project.doc["step_unit"] = 200
project.doc["equil_time"] = 100
project.doc["min_periods"] = 20
project.doc["dumps"] = 40
project.doc["period_times"] = [30.0, 100.0, 300.0, 1000.0]
project.doc["max_shears"] = [0.01, 0.02, 0.03, 0.05, 0.08, 0.12]

In [12]:
for job in project:
    print(job)

634e14bc75836df75158fb15f379d8bb
cfa1e4c0007fd65d020f809715a6f0b4
ecce68c50e28a33684826f28780bf6e9
1083a2a09eec2c6cf215ebc7a9706cac
6976a7c10ada4159010403f8027f3ea5
9158e73b2e4c9fd1a565e1e5d1bc3af5


In [10]:
# HTL

N = 2**15  # 32758
rhos = [1.2]
init_temps = [1.5]
pot = "KA_LJ"
prepared = "HTL"

for replica in range(6):
    for init_temp, rho in zip(init_temps, rhos):
        sp = {"N": N, "replica": replica, "rho": rho, "pot": pot, "prep": prepared}
        job = project.open_job(sp)
        if "init" not in job.doc:
            tg = tgs[pot]
            job.init()
            job.doc["init"] = True
            job.doc["seed"] = project.doc["avail_seed"]
            job.doc["init_temp"] = init_temp
            job.doc["tg"] = tg
            job.doc["temps"] = [tg/4.0, tg/2.0, tg*3.0/4.0, tg]
            project.doc["avail_seed"] += 1

/home/ian/Projects/work/monk/.venv/lib/python3.10/site-packages/signac/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


In [29]:
# for job in project.find_jobs({"prep": "ESL"}):
#     print(job, job.sp)
#     job.remove()

In [30]:
# ESL

N = 2**15  # 32758
rhos = [1.2]
pot = "KA_LJ"
prepared = "ESL"
replicas = [0, 2, 4, 5, 7, 8]

for replica in replicas:
    matching_jobs = origin.find_jobs({"pot": pot, "replica": replica})
    assert len(matching_jobs) == 1
    origin_job = matching_jobs.next()
    sim = sorted(glob.glob(origin_job.fn("fine/equil_*.gsd")))[0]
    print(sim)
    assert float(utils.extract_between(sim, "equil_temp-", ".gsd")) == 0.3

    for rho in rhos:
        sp = {"N": N, "replica": replica, "rho": rho, "pot": pot, "prep": prepared}
        job = project.open_job(sp)
        if "init" not in job.doc:
            tg = tgs[pot]
            job.init()
            job.doc["init"] = True
            job.doc["seed"] = project.doc["avail_seed"]
            job.doc["init_temp"] = 0.3
            job.doc["tg"] = tg
            job.doc["temps"] = [tg/4.0, tg/2.0, tg*3.0/4.0, tg]
            project.doc["avail_seed"] += 1
            shutil.copy(sim, job.fn("start-file.gsd"))
            

/media/ian/Data2/monk/2d-esl/workspace/a03bdc2a5be70a1e699a3f825fb8877b/fine/equil_temp-0.300.gsd


/home/ian/Projects/work/monk/.venv/lib/python3.10/site-packages/signac/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


/media/ian/Data2/monk/2d-esl/workspace/127b38ea46bce9a180ac54e7adf90bf5/fine/equil_temp-0.300.gsd
/media/ian/Data2/monk/2d-esl/workspace/a0f72fa2c65b6e380a9e3b21848079ed/fine/equil_temp-0.300.gsd
/media/ian/Data2/monk/2d-esl/workspace/195c4fa433b01286ebe2ce7c821cb9e1/fine/equil_temp-0.300.gsd
/media/ian/Data2/monk/2d-esl/workspace/eb25523e79e0f62b126ad67ec75f5ca7/fine/equil_temp-0.300.gsd
/media/ian/Data2/monk/2d-esl/workspace/503e0376bd64085f1c22ed91ae48db88/fine/equil_temp-0.300.gsd


In [ ]:
# initialize projects using data from the prior project

# some data should be copied, others probably don't need to be